In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn.model_selection import train_test_split
import torch.optim as optim

In [3]:
# Load the fine-tuned model and tokenizer
model = BartForConditionalGeneration.from_pretrained("/kaggle/input/notebook67071bc6e0/fine_tuned_bart_model")
tokenizer = BartTokenizer.from_pretrained("/kaggle/input/notebook67071bc6e0/fine_tuned_bart_model")

In [9]:
# Load BART tokenizer and model
pre_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
pre_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
pre_model.to(device)
import torch

# Check if CUDA is available and set the device accordingly
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the appropriate device
model.to(device)

def summarizeText(text, mymodel):
    # Move input tensors to the same device as the model
    text_encoding = tokenizer(
        text,
        max_length=1000,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    ).to(device)

    generated_ids = mymodel.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask'],
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)

In [10]:
test_data=pd.read_csv('/kaggle/input/notebook67071bc6e0/val.csv')
test = test_data.sample(5)
for i in range(len(test)):
    sample_row = test.iloc[i]
    text = sample_row['text']
    print('Text : ', text)
    print('\n')
    print('Summary (Ground truth) : ',sample_row['summary'])
    print('\n')
    summary_without_tuning = summarizeText(text, pre_model)
    print("Summary before tuning", summary_without_tuning)
    print('\n')
    summary_with_tuning = summarizeText(text, model)
    print("Summary after tuning", summary_with_tuning)
    print('\n')
    print('-------------------------------------------------------------------------------------------------')

Text :  لا تردي على أية رسائل نصية لا تتعلق مباشرة بتجمع العائلة أو الرسائل الإيجابية أو المواضيع الطبيعية المعتادة. إذا وصلتك رسائل توضح أنها غاضبة من الأشياء التي حدثت لها وانزعاجها من شيء قمت به أو أي ثرثرة لا فائدة منها، فاتركيها بلا رد. إذا شعرت بالغضب ورغبت في إرسال رد أو توبيخ أو تبرير، لا تفعلي ذلك، وإنما تجاهلي غضبك وتعاملي معه كإنه إشارة تحذير للنوم. لن يفيد تبادل الرسائل النصية في تلك اللحظة سوى في زيادة الغضب والحنق في كلا الجانبين. إذا كانت أخت زوجك درامية إلى ذلك الحد، فمن الممكن أن تعكس شبكتها الاجتماعية بحثها الدائم عن الاهتمام. يمكنك بكل سهولة الانخراط في موجات غضبها والدرامية بمجرد رؤيتك لمنشوراتها على فيسبوك أو تويتر.  إذا أرسلت لك طلب صداقة، فهناك الكثير من الأشياء التي يمكنك القيام بها.  أولا: تجاهلي الطلب ببساطة، وعندما تسألك عن الأمر أخبريها بأنك لا تستخدمي مواقع التواصل الاجتماعي كثيرا إلا لتبادل المعلومات المهمة أو أنك لا تستخدميها على الإطلاق. ثانيا: الرد بالشكر بقول "شكرا ولكني لا أقبل طلبات جديدة في الوقت الراهن بسبب الانشغال أو الخصوصية وما إلى ذلك"، يمكنك 

In [11]:
pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=5016a656035beff09bc9c815efca5272bf23dc1ce0efc681c715f2ccbcde8c2e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [20]:
from rouge_score import rouge_scorer
test=test_data.sample(100)
# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store individual ROUGE scores
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

# Iterate through test samples and corresponding summaries
for i in range(len(test)):
    sample_row = test.iloc[i]
    text = sample_row['text']
    summary = sample_row['summary']

    summary_with_tuning = summarizeText(text, model)

    # Compute ROUGE scores for each sample
    scores = scorer.score(summary, summary_with_tuning)

    # Store individual ROUGE scores
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

# Compute average ROUGE scores
avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print("Average ROUGE-1 F1 Score:", avg_rouge1)
print("Average ROUGE-2 F1 Score:", avg_rouge2)
print("Average ROUGE-L F1 Score:", avg_rougeL)

Average ROUGE-1 F1 Score: 0.015507736943907155
Average ROUGE-2 F1 Score: 0.002173913043478261
Average ROUGE-L F1 Score: 0.015507736943907155
